 # QQQ数据分析

In [1]:
#imports

import yfinance as yf
import pandas as pd
import datetime as dt
import getdata as gd
import warnings


warnings.filterwarnings('ignore')

In [2]:
print('This notebook was refreshed at %s'%(dt.datetime.now()))

This notebook was refreshed at 2022-01-11 18:26:00.349873


本笔记分析QQQ的数据以回答以下问题：
1. 当前的估值是否合理？
2. 中短期趋势如何？(未完待续)

In [3]:
#init

pd.options.display.float_format = '{:,.2f}'.format
df_port=pd.read_csv('../data/share/qqq_holdings.csv')


## 成分股基本面监测

In [4]:
df_stock=gd.get_port_holding_info(df_port)
df_stock.to_csv('../data/share/qqq_holidngs_info.csv')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [5]:
df_stock=pd.read_csv('../data/share/qqq_holidngs_info.csv')
df_metric=gd.get_port_holding_metric(df_stock)

In [9]:
df_metric.reindex(columns=['ticker','weight','price','drawdown','pcf','pe_ttm',
                'pe_fwd','est_eps_growth','peg','roe','cf_margin','net_margin'])\
                .sort_values(by='drawdown')\
                .style.format(precision=2,thousands=',',formatter={'weight':'{:.3f}',
                                                                    'drawdown':'{:.1f}%',
                                                                    'est_eps_growth':'{:.1f}%',
                                                                    'gross_margin':'{:.1f}%',
                                                                    'cf_margin':'{:.1f}%',
                                                                    'net_margin':'{:.1f}%',

                                                                
                                                                
                                                                    
                                                                    })

,ticker,weight,price,drawdown,pcf,pe_ttm,pe_fwd,est_eps_growth,peg,roe,cf_margin,net_margin
100,PTON,0.001,34.37,-79.5%,-10.20,nan,-40.92,-60.7%,0.21,-38.35,-27.3%,-15.6%
98,PDD,0.002,56.99,-71.9%,16.65,nan,42.21,-3957.1%,-11.39,-0.47,29.3%,-0.2%
74,ZM,0.003,172.56,-61.2%,28.64,45.86,39.49,16.1%,2.59,34.64,45.9%,29.3%
92,DOCU,0.002,136.23,-56.1%,56.05,nan,63.36,-471.5%,1.52,-33.64,24.6%,-5.7%
88,BIDU,0.002,152.36,-55.2%,15.76,25.19,17.04,47.8%,67.70,5.39,17.6%,11.2%
32,MRNA,0.006,233.70,-51.8%,8.19,14.00,8.39,66.8%,0.49,109.61,97.8%,59.7%
40,MELI,0.004,"1,053.31",-46.9%,102.03,667.50,119.69,457.7%,19.99,8.22,8.3%,1.3%
87,BIIB,0.002,235.31,-43.3%,14.20,23.03,12.73,80.9%,-1.87,19.47,21.9%,13.9%
15,PYPL,0.015,182.95,-40.7%,36.29,43.99,34.65,27.0%,1.99,24.30,24.1%,20.1%
59,ATVI,0.003,63.11,-38.9%,16.99,18.72,16.61,12.7%,1.33,16.88,32.0%,29.2%


## 整体基本面分析

若把整个ETF当成一个整体，按成分股权重计算，各项基本面指标如下：

In [7]:
df_port_metric=gd.get_port_metric(df_metric)

In [10]:
df_port_metric.columns=['QQQ']
df_port_metric

,QQQ
market_cap,"997,802,826,488.86"
revenue,"133,862,618,184.98"
revenue_growth,24.76
ps,7.45
gross_profit,"58,036,776,362.81"
pgp,17.19
gross_margin,43.36
cashflow,"37,352,303,534.54"
pcf,26.71
cf_rate,3.74
